In [8]:
!python --version

Python 3.10.15


In [6]:
%conda install pytorch torchvision -c pytorch-nightly

Channels:
 - pytorch-nightly
 - defaults
 - conda-forge
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/homebrew/Caskroom/miniconda/base/envs/met-museum-medieval

  added / updated specs:
    - pytorch
    - torchvision


The following NEW packages will be INSTALLED:

  blas               pkgs/main/osx-arm64::blas-1.0-openblas 
  brotli-python      pkgs/main/osx-arm64::brotli-python-1.0.9-py310h313beb8_8 
  charset-normalizer pkgs/main/noarch::charset-normalizer-3.3.2-pyhd3eb1b0_0 
  filelock           pkgs/main/osx-arm64::filelock-3.13.1-py310hca03da5_0 
  freetype           pkgs/main/osx-arm64::freetype-2.12.1-h1192e45_0 
  giflib             pkgs/main/osx-arm64::giflib-5.2.2-h80987f9_0 
  idna               pkgs/main/osx-arm64::idna-3.7-py310hca03da5_0 
  jinja2             pkgs/main/osx-arm64::jinja2-3.1.4-py310hca03da5_0 
  jpeg               pkgs/main/osx-arm64::jpeg-9e-h80987f9_3 
  lcms2              pkgs/main/osx-arm64::lcms2-2.1

In [4]:
%pip install pandas numpy pillow
%pip install scikit-learn
%pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import numpy as np
import torch
from torchvision import transforms, models
from PIL import Image
from pathlib import Path
import plotly.express as px
from sklearn.manifold import TSNE

ImportError: cannot import name 'Tensor' from 'torch' (unknown location)

In [37]:
import plotly.io as pio
pio.renderers.default = 'notebook'

In [2]:
df = pd.read_csv('met_objects.csv')
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

Dataset shape: (7136, 11)

First few rows:
   objectID accessionNumber  \
0    157612   2009.300.2750   
1    462739        04.2.818   
2    462740        04.2.819   
3    462741        04.2.820   
4    462742        04.2.821   

                                        primaryImage  \
0  https://images.metmuseum.org/CRDImages/md/orig...   
1  https://images.metmuseum.org/CRDImages/md/orig...   
2  https://images.metmuseum.org/CRDImages/md/orig...   
3  https://images.metmuseum.org/CRDImages/md/orig...   
4  https://images.metmuseum.org/CRDImages/md/orig...   

                                   primaryImageSmall objectName  \
0  https://images.metmuseum.org/CRDImages/md/web-...    Orphrey   
1  https://images.metmuseum.org/CRDImages/md/web-...    Solidus   
2  https://images.metmuseum.org/CRDImages/md/web-...    Solidus   
3  https://images.metmuseum.org/CRDImages/md/web-...    Solidus   
4  https://images.metmuseum.org/CRDImages/md/web-...    Solidus   

                              

In [17]:
# remove rows with missing image URLs
df = df.dropna(subset=['primaryImage'])

# show length of dataset after removing rows with missing image URLs
print("Dataset shape after removing rows with missing image URLs:", df.shape)

Dataset shape after removing rows with missing image URLs: (6917, 11)


In [3]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [7]:
device = (
    "mps" 
    if torch.backends.mps.is_available() 
    else "cpu"
)
print(f"Using device: {device}")

Using device: mps


In [8]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
# Remove the final classification layer
model = torch.nn.Sequential(*list(model.children())[:-1])
model = model.to(device)
model.eval()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /Users/zach/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100.0%


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [18]:
img_dir = Path('images')
embeddings = []
processed_metadata = []
batch_size = 32

for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_embeddings = []
    
    for _, row in batch_df.iterrows():
        # only if image exists
        if (img_dir / f"{row['objectID']}.jpg").exists():
            try:
                # Load and preprocess image
                img_path = img_dir / f"{row['objectID']}.jpg"
                image = Image.open(img_path).convert('RGB')
                image_tensor = preprocess(image).unsqueeze(0).to(device)
                
                # Generate embedding
                with torch.no_grad():
                    embedding = model(image_tensor)
                    embedding = embedding.squeeze().cpu().numpy()
                    
                batch_embeddings.append(embedding)
                processed_metadata.append({
                    'objectID': row['objectID'],
                    'title': row['title']
                    # Add any other metadata fields you want to keep
                })
                
            except Exception as e:
                print(f"Error processing {row['objectID']}: {e}")
                continue
        else:
            print(f"Image not found for {row['objectID']}")
    
    embeddings.extend(batch_embeddings)
    print(f"Processed {len(embeddings)} images")

Processed 32 images
Processed 64 images
Processed 96 images
Processed 128 images
Processed 160 images
Processed 192 images
Image not found for 463201
Processed 223 images
Processed 255 images
Processed 287 images
Processed 319 images
Processed 351 images
Processed 383 images
Processed 415 images
Processed 447 images
Processed 479 images
Processed 511 images
Processed 543 images
Processed 575 images
Processed 607 images
Processed 639 images
Processed 671 images
Processed 703 images
Processed 735 images
Processed 767 images
Processed 799 images
Processed 831 images
Processed 863 images
Processed 895 images
Processed 927 images
Processed 959 images
Image not found for 464931
Processed 990 images
Processed 1022 images
Processed 1054 images
Processed 1086 images
Processed 1118 images
Processed 1150 images
Processed 1182 images
Processed 1214 images
Processed 1246 images
Processed 1278 images
Processed 1310 images
Processed 1342 images
Processed 1374 images
Processed 1406 images
Processed 14

In [19]:
embeddings

[array([0.02943092, 0.00421154, 0.39779222, ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 array([0.06907681, 0.00924976, 0.        , ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 array([0.03793921, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 array([0.15432765, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 array([9.8160398e-01, 2.1305862e-04, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00], dtype=float32),
 array([1.5415555, 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ], dtype=float32),
 array([0.00210683, 0.        , 0.02662036, ..., 0.        , 0.        ,
        0.01964532], dtype=float32),
 array([0.22840597, 0.07208687, 0.        , ..., 0.        , 0.        ,
        0.        ], dtype=float32),
 array([0.20660858, 0.13507883, 0.        , ..., 0.        , 0.        ,
        0.        ], dtype=float32)

In [20]:
processed_metadata

[{'objectID': 157612, 'title': 'Orphrey'},
 {'objectID': 462739, 'title': 'Solidus'},
 {'objectID': 462740, 'title': 'Gold Solidus of Theodosius II (408–50)'},
 {'objectID': 462741, 'title': 'Solidus'},
 {'objectID': 462742,
  'title': 'Solidus of Heraclius and Heraclius Constantine'},
 {'objectID': 462743,
  'title': 'Solidus of Heraclius, Heraclius Constantine, and Heraclonas'},
 {'objectID': 462744, 'title': 'Panel with John the Baptist and other saints'},
 {'objectID': 462765, 'title': 'Solidus'},
 {'objectID': 462766, 'title': 'Solidus'},
 {'objectID': 462767, 'title': 'Solidus'},
 {'objectID': 462768, 'title': 'Gold Solidus of Constans II (641–68)'},
 {'objectID': 462769, 'title': 'Solidus'},
 {'objectID': 462770, 'title': 'Gold Tremissis of Emperor Justinian I'},
 {'objectID': 462771, 'title': 'Olearium'},
 {'objectID': 462773, 'title': 'Chest Front'},
 {'objectID': 462776, 'title': 'Pietà (Lamentation)'},
 {'objectID': 462778, 'title': 'Chest Front'},
 {'objectID': 462780, 'tit

In [21]:
embeddings_array = np.stack(embeddings)
print(f"\nFinal embeddings shape: {embeddings_array.shape}")


Final embeddings shape: (4389, 2048)


In [22]:
print("\nReducing dimensions with t-SNE...")
tsne = TSNE(n_components=2, random_state=42)
reduced_embeddings = tsne.fit_transform(embeddings_array)


Reducing dimensions with t-SNE...


In [24]:
viz_df = pd.DataFrame({
    'x': reduced_embeddings[:, 0],
    'y': reduced_embeddings[:, 1],
    'title': [m['title'] for m in processed_metadata],
    'objectID': [m['objectID'] for m in processed_metadata]
})

# save the reduced embeddings to a CSV file
viz_df.to_csv('reduced_embeddings.csv', index=False)

In [10]:
viz_df = pd.read_csv('reduced_embeddings.csv')

fig = px.scatter(
    viz_df,
    x='x',
    y='y',
    hover_data=['title', 'objectID'],
    title='Artwork Embeddings Visualization'
)

In [27]:
np.save('embeddings.npy', embeddings_array)
viz_df.to_csv('processed_metadata.csv', index=False)
fig.write_html('artwork_embeddings.html')

In [11]:
fig.show()